In [55]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
df = pd.read_csv('data_csv.csv', sep=';')
df_old=df
df_old.head()

,sex,age,growth,birth,EGEmath,EGEruss,EGEphis,s1,s2,s3,...,yes3,yes4,yes5,yes6,yes7,hostel,duration,course,coin,stones
0,0,19,173,14.05.1996,56,67,47,3.6,3.6,3.50,...,0,0,0,0,0,0,15,2,1,2
1,1,19,182,22.08.1996,50,50,49,3.0,3.0,4.00,...,1,0,0,0,0,0,90,2,0,0
2,1,20,193,13.10.1995,67,62,58,4.5,4.0,3.25,...,1,0,0,0,0,0,15,2,0,0
3,1,29,170,01.12.1986,100,100,100,5.0,5.0,5.00,...,1,1,1,1,1,0,30,5,0,2
4,0,20,160,01.09.1995,70,54,46,3.0,3.0,3.00,...,0,0,0,0,0,0,20,2,0,1


In [56]:
df=df_old[['sex','course','coin','stones','yes1']]
df.head()

,sex,course,coin,stones,yes1
0,0,2,1,2,1
1,1,2,0,0,0
2,1,2,0,0,1
3,1,5,0,2,1
4,0,2,0,1,0


In [57]:
# не будем дя простоты разбивать на тренировочную и тестовую выборку

# a person for prediction
man= pd.DataFrame([[1,2,1,1,np.nan]], columns=list(df.columns.values))
man

,sex,course,coin,stones,yes1
0,1,2,1,1,NaN


In [58]:
print(df[df['sex']==1].shape[0])
print(df[df['coin']==1].shape[0])
print(df[df['course']==2].shape[0])
print(df[df['stones']==1].shape[0])
print(df[df['yes1']==1].shape[0])

31
18
34
32
38


# Bayes formula
Pr(Y|A)=Pr(A|Y)Pr(Y)/Pr(A)

Нужно найти:
Pr(yes1=1)=? Это будет Pr(Y) для формулы Байеса
Pr(sex=1|yes1=1)=? Это будет Pr(А|Y) для формулы Байеса

То же самое для класса 0:

Pr(yes1=0)=? Это будет Pr(Y) для формулы Байеса
Pr(sex=1|yes1=0)=? Это будет Pr(А|Y) для формулы Байеса


То же самое нужно сделать и для остальных признаков:

Pr(coin=1|yes1=1)=?

Pr(stones=1|yes1=1)=?

Pr(course=2|yes1=1)=?

Pr(coin=1|yes1=0)=?

Pr(stones=1|yes1=0)=?

Pr(course=2|yes1=0)=?


In [59]:
# так считаем вероятности Pr(yes1=1),Pr(yes1=0)
pr_yes1_1=df[df['yes1']==1].shape[0]/df.shape[0]
pr_yes1_0=df[df['yes1']==0].shape[0]/df.shape[0]
print(pr_yes1_1,pr_yes1_0)

0.6333333333333333 0.36666666666666664


In [60]:
# считаем Pr(sex=1|yes1=0),Pr(sex=1|yes1=1) (!!!) их сумма не обязана быть равной =1
pr_sex_1=df[np.logical_and(df['yes1']==1,df['sex']==1)].shape[0]/df[df['yes1']==1].shape[0]
pr_sex_0=df[np.logical_and(df['yes1']==0,df['sex']==1)].shape[0]/df[df['yes1']==0].shape[0]
print(pr_sex_1,pr_sex_0)

0.4473684210526316 0.6363636363636364


In [62]:
# аналогично считаем вероятности для бинарного признака coin
pr_coin_1=df[np.logical_and(df['yes1']==1,df['coin']==1)].shape[0]/df[df['yes1']==1].shape[0]
pr_coin_0=df[np.logical_and(df['yes1']==0,df['coin']==1)].shape[0]/df[df['yes1']==0].shape[0]
print(pr_coin_1,pr_coin_0)

0.34210526315789475 0.22727272727272727


In [63]:
#Внимание! А теперь признак "курс". Он не бинарный. Но тут все аналогично
pr_course_1=df[np.logical_and(df['yes1']==1,df['course']==2)].shape[0]/df[df['yes1']==1].shape[0]
pr_course_0=df[np.logical_and(df['yes1']==0,df['course']==2)].shape[0]/df[df['yes1']==0].shape[0]
print(pr_course_1,pr_course_0)

0.631578947368421 0.45454545454545453


In [64]:
#Внимание! А теперь признак "stones". Он не бинарный. Но тут все аналогично
pr_stones_1=df[np.logical_and(df['yes1']==1,df['stones']==1)].shape[0]/df[df['yes1']==1].shape[0]
pr_stones_0=df[np.logical_and(df['yes1']==0,df['stones']==1)].shape[0]/df[df['yes1']==0].shape[0]
print(pr_stones_1,pr_stones_0)

0.5789473684210527 0.45454545454545453


In [65]:
# Все готово для вычисления числителя в формуле Байеса
pr_1_numerator=pr_yes1_1*pr_sex_1*pr_coin_1*pr_course_1*pr_stones_1
pr_0_numerator=pr_yes1_0*pr_sex_0*pr_coin_0*pr_course_0*pr_stones_0
print(pr_1_numerator,pr_0_numerator)

0.03544248432716139 0.010956674179814675


In [66]:
#А как получить окончательный ответ?
R=pr_1_numerator/pr_0_numerator
print('не сдаст:',1/(R+1))
print('сдаст:',1-(1/(R+1)))

не сдаст: 0.23613950193013417
сдаст: 0.7638604980698658


# Домашка
Написанный выше алгоритм Байеса достаточно неэффективен: мы фактически копи-пастом создали несколько ячеек с похожим кодом. Если бы таблица содержала бы 100 нецелевых признаков, то нам бы пришлось писать 100 ячеек с кодом. (((((((

Короче говоря, нужно написать цикл, который

1) бежит по нецелевым признакам

2) для каждого нецелевого признака считает пару вероятностей по формулам выше

3) найденные в п.2) вероятности сохранить в списки

После цикла: нужно пробежатьс по полученным спискам и вычислить 

1) сначала числители для формулы Байеса

2) а потом и окончателные вероятности

П.С. В цикл можете не перебирать ЦЕЛЕВОЙ признак, а вычислить числа pr_yes1_1, pr_yes1_0 по нашим формулам.